# WEB幹事スクレイピング

In [ ]:
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

## 0. 自動操作用ブラウザの立ち上げ

In [ ]:
options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

## 1. 検索結果1ページ目から個別ページのリンクを取得する

今回は「秋田県」を検索条件に指定した場合を例にハンズオン形式で進めていきます。

検索条件の指定ページ: https://web-kanji.com/search/

In [ ]:
url = 'https://web-kanji.com/search/akita'
driver.get(url)

companies-item をクラス名に持つタグを抽出します

```html
<a href="https://web-kanji.com/companies/xxx01" class="companies-item">会社1のコンテンツ</a>
<a href="https://web-kanji.com/companies/xxx02" class="companies-item">会社2のコンテンツ</a>
<a href="https://web-kanji.com/companies/xxx03" class="companies-item">会社3のコンテンツ</a>
```

In [ ]:
# リスト
company_items = driver.find_elements(By.CSS_SELECTOR, '.companies-item')

In [ ]:
print(len(company_items))
print(company_items[0].get_attribute('href'))

In [ ]:
single_company_urls = [item.get_attribute('href') for item in company_items]

### 関数化

In [ ]:
def fetch_single_company_urls(driver, url):
    """
    パラメータ:
        driver: WebDriver
        url: 検索結果の一覧ページ
    返却値:
        single_company_urls: 制作会社個別ページのURLリスト

    urlに載っている制作会社個別ページのURLを取得する
    """
    driver.get(url)
    sleep(1)
    company_items = driver.find_elements(By.CSS_SELECTOR, '.companies-item')
    single_company_urls = [item.get_attribute('href') for item in company_items]
    return single_company_urls


## 2. 個別ページの会社情報を抽出する

In [ ]:
url = single_company_urls[0]
driver.get(url)

会社情報のHTML構造

```html
<dl class="company-data is-narrow">
    <dt>会社名</dt>
    <dd>株式会社xxx</dd>

    <dt>設立</dt>
    <dd>2000年0月</dd>
    
    <dt>代表</dt>
    <dd>山田 太郎</dd>
    
    <dt>本社所在地</dt>
    <dd>
        〒 123-4567<br>
        秋田県xxx市yyy番地
    </dd>
    
    <dt>資本金</dt>
    <dd>
        00万円
    </dd>
    
    <dt>URL</dt>
    <dd>
        <a class="link js-forced-ad is-wordbreak" href="http://example.co.jp" target="_blank" rel="noopener">
            http://example.co.jp
        </a>
    </dd>
</dl>

```

In [ ]:
dl_element = driver.find_element(By.CSS_SELECTOR, '.company-data.is-narrow')
#dl_element = driver.find_elements(By.CSS_SELECTOR, '.company-data.is-narrow')[0] と同じ

dt_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dt')
dd_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dd')

datum = {}
for dt, dd in zip(dt_elements, dd_elements):
    datum[dt.text] = dd.text.replace('\n', ' ')
datum

### 関数化

In [ ]:
def extract_company_data(driver, url):
    """
    パラメータ:
        driver: WebDriver
        url: 個別ページURL
    返却値:
        data: 制作会社の会社情報のdict
              (例) 
              {
                  'WEB幹事_URL': 'http://web-kanji....', 
                  '会社名': 'xxx株式会社', 
                  '代表': '山田 太郎', 
                  'URL': 'http://xxx.co.jp/', 
                  ...
              }
    """
    driver.get(url)
    sleep(1)
    dl_element = driver.find_element(By.CSS_SELECTOR, '.company-data.is-narrow')
    dt_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dt')
    dd_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dd')

    data = {}
    data['WEB幹事_URL'] = url
    for dt, dd in zip(dt_elements, dd_elements):
        data[dt.text] = dd.text.replace('\n', ' ')
    return data

## 3. 20社の会社概要を取得する ※ 今回は3社

ただし、今回は皆さん同時に20件アクセスするとサーバに負荷をかけてしまうので3件とします。

In [ ]:
search_url = 'https://web-kanji.com/search/akita'
single_company_urls = fetch_single_company_urls(driver, search_url)

In [ ]:
# 3社のみにフィルタ
print(len(single_company_urls))
single_company_urls = single_company_urls[:3]
print(len(single_company_urls))

In [ ]:
company_data = [extract_company_data(driver, url) for url in single_company_urls]

In [ ]:
df = pd.DataFrame(company_data).fillna('')
df

## 4. 検索結果の2ページ目以降に掲載されている制作会社個別ページも取得する

### 方法1 (易しい、手作業多い)

検索結果のページネーションURLを直書きで指定する

In [ ]:
search_urls = [
    'https://web-kanji.com/search/akita',
    'https://web-kanji.com/search/akita/page/2',
]

In [ ]:
single_company_urls = []
for url in search_urls:
    urls = fetch_single_company_urls(driver, url)
    single_company_urls.extend(urls)

print(len(single_company_urls))
single_company_urls = single_company_urls[:3]
print(len(single_company_urls))

In [ ]:
company_data = [extract_company_data(driver, url) for url in single_company_urls]
df = pd.DataFrame(company_data).fillna('')
df

### 方法2 (難しい、手作業少ない)

In [ ]:
search_url = 'https://web-kanji.com/search/akita'
driver.get(search_url)

In [ ]:
search_urls = []
current_url = search_url
while True:
    driver.get(current_url)
    sleep(1)
    search_urls.append(current_url)
    try:
        next_a = driver.find_element(By.CSS_SELECTOR, '.pagination-item > a[rel="next"]')
    except NoSuchElementException:
        break
    current_url = next_a.get_attribute('href')

### 関数化

In [ ]:
def fetch_search_pagination_urls(driver, search_url):
    """
    パラメータ:
        driver: WebDriver
        search_url: 検索結果１ページ目のURL
    返却値:
        search_urls: 検索条件のページネーションURLリスト
    
    検索結果のページネーションを自動で検知してページネーションのURLをすべて取得する
    """
    search_urls = []
    current_url = search_url
    while True:
        driver.get(current_url)
        sleep(1)
        search_urls.append(current_url)
        try:
            next_a = driver.find_element(By.CSS_SELECTOR, '.pagination-item > a[rel="next"]')
        except NoSuchElementException:
            break
        current_url = next_a.get_attribute('href')
    return search_urls

In [ ]:
search_urls = fetch_search_pagination_urls(driver, search_url)
search_urls

In [ ]:
single_company_urls = []
for url in search_urls:
    urls = fetch_single_company_urls(driver, url)
    single_company_urls.extend(urls)
print(len(single_company_urls))
single_company_urls = single_company_urls[:3]

In [ ]:
company_data = [extract_company_data(driver, url) for url in single_company_urls]
df = pd.DataFrame(company_data).fillna('')
df

## 5. 抽出したデータをファイルに書き出す

### tsvフォーマットで書き出す

In [ ]:
df.to_csv('company_data.tsv', sep='\t')